# Central Weather Bureau Scrapper

Create a weather view for next 6 hours in each cities in Taiwan.

In [1]:
import requests
import json

%precision 2

'%.2f'

In [2]:
response = requests.get("https://opendata.cwb.gov.tw/api/v1/rest/datastore/F-C0032-001?Authorization=CWB-4F680A3D-49B6-4B48-BA1B-308A6C464DEF")

In [3]:
response.reason

'OK'

In [4]:
json_data = response.json()

In [6]:
from datetime import datetime

In [7]:
location = json_data['records']['location']

lst = []
for i in location:
    city = i['locationName']                                                 # city 
    
    date_time_str = i['weatherElement'][0]['time'][0]['startTime']           # time of record
    date_time = datetime.strptime(date_time_str,'%Y-%m-%d %H:%M:%S')
    
    wx6 = i['weatherElement'][0]['time'][0]['parameter']['parameterName']    # weather phonenmon
    pop6 = i['weatherElement'][1]['time'][0]['parameter']['parameterName']   # chance of rain
    mint6 = i['weatherElement'][2]['time'][0]['parameter']['parameterName']  # lowest temperature
    ci6 = i['weatherElement'][3]['time'][0]['parameter']['parameterName']    # comfortable index
    maxt6 = i['weatherElement'][4]['time'][0]['parameter']['parameterName']  # highest temperature
        
    lst.append((date_time, city, wx6, int(maxt6),  int(mint6), int(pop6)/100))

In [8]:
lst

[(datetime.datetime(2023, 7, 11, 12, 0), '嘉義縣', '多雲午後短暫雷陣雨', 34, 31, 0.50),
 (datetime.datetime(2023, 7, 11, 12, 0), '新北市', '晴午後短暫雷陣雨', 34, 31, 0.60),
 (datetime.datetime(2023, 7, 11, 12, 0), '嘉義市', '多雲午後短暫雷陣雨', 34, 30, 0.70),
 (datetime.datetime(2023, 7, 11, 12, 0), '新竹縣', '晴時多雲', 33, 31, 0.20),
 (datetime.datetime(2023, 7, 11, 12, 0), '新竹市', '晴時多雲', 32, 30, 0.10),
 (datetime.datetime(2023, 7, 11, 12, 0), '臺北市', '多雲午後短暫雷陣雨', 35, 31, 0.50),
 (datetime.datetime(2023, 7, 11, 12, 0), '臺南市', '多雲時晴', 32, 30, 0.20),
 (datetime.datetime(2023, 7, 11, 12, 0), '宜蘭縣', '晴時多雲', 33, 30, 0.10),
 (datetime.datetime(2023, 7, 11, 12, 0), '苗栗縣', '多雲時晴', 33, 31, 0.20),
 (datetime.datetime(2023, 7, 11, 12, 0), '雲林縣', '多雲午後短暫雷陣雨', 34, 30, 0.60),
 (datetime.datetime(2023, 7, 11, 12, 0), '花蓮縣', '多雲短暫陣雨', 32, 29, 0.30),
 (datetime.datetime(2023, 7, 11, 12, 0), '臺中市', '晴午後短暫雷陣雨', 34, 32, 0.50),
 (datetime.datetime(2023, 7, 11, 12, 0), '臺東縣', '多雲', 31, 29, 0.10),
 (datetime.datetime(2023, 7, 11, 12, 0), '桃園市', '

In [9]:
# location = json_data['records']['location']

# lst2 = []
# for i in location:
#     for t in range(3):
#         city = i['locationName']    # city name
#         wx6 = i['weatherElement'][0]['time'][t]['parameter']['parameterName']    # weather phonenmon
#         pop6 = i['weatherElement'][1]['time'][t]['parameter']['parameterName']   # chance of rain
#         mint6 = i['weatherElement'][2]['time'][t]['parameter']['parameterName']  # lowest temperature
#         ci6 = i['weatherElement'][3]['time'][t]['parameter']['parameterName']    # comfortable index
#         maxt6 = i['weatherElement'][4]['time'][t]['parameter']['parameterName']  # highest temperature

#         lst2.append((city, wx6, int(maxt6),  int(mint6), int(pop6)/100))

In [10]:
import pymysql

conn = pymysql.connect(
    host='localhost',
    user='root',
    password='Leoyen-1998',
    db='weather',
    cursorclass=pymysql.cursors.DictCursor
)

In [11]:
try:
    with conn.cursor() as cursor:
        
        sql = """INSERT INTO weather_6hr (date, city, weather, temperature_highest, temperature_lowest, chance_of_rain) 
    VALUES(%s, %s, %s, %s, %s, %s)"""
        
        for entry in lst:
            cursor.execute(sql, entry)
            
    conn.commit()
    print("Record inserted successfully")
    
finally:
    conn.close()

Record inserted successfully
